 LABEL ENCODING method and FREQUENCY ENCODING method resulted in the same higher accuracy (0.9283), while the ONEHOT ENCODING method had a lower accuracy (0.8733), we can finalize one of the better-performing methods.

Since both the first and third methods achieved the same accuracy, we can choose either based on additional factors such as interpretability and simplicity.

Final Chosen Method:
Label Encoding and Count Vectorization

Reasons for choosing this method:


High Accuracy : Achieved a high accuracy of 0.9283.

Simplicity : Easy to implement and understand.

Efficiency : Count Vectorization is computationally less intensive than TF-IDF.

Step 1: Load the Dataset

We start by loading the dataset using pandas, a powerful data manipulation library. This step is crucial to understand the structure and contents of the data.

In [1]:
import pandas as pd

# Load the dataset
file_path = '/content/HateSpeechDetection (preprocessed).csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset
print(data.head())


  Platform                                            Comment  Hateful
0   Reddit  Damn I thought they had strict gun laws in Ger...        0
1   Reddit  I dont care about what it stands for or anythi...        0
2   Reddit                  It's not a group it's an idea lol        0
3   Reddit                          So it's not just America!        0
4   Reddit  The dog is a spectacular dancer considering he...        0


Step 2:


Encode the 'Platform' Column using Label Encoding

Label Encoding transforms categorical values into integers. This is particularly useful for machine learning algorithms that cannot work with categorical data directly.

In [2]:
from sklearn.preprocessing import LabelEncoder

# Label Encode the 'Platform' column
label_encoder = LabelEncoder()
data['Platform_encoded'] = label_encoder.fit_transform(data['Platform'])

# Display the encoded platform data
print(data[['Platform', 'Platform_encoded']].head())


  Platform  Platform_encoded
0   Reddit                 1
1   Reddit                 1
2   Reddit                 1
3   Reddit                 1
4   Reddit                 1


Purpose:
To convert the Platform column into a numerical format.

Label Encoding:
Assigns a unique integer to each category.

For example, if Platform has values like Reddit, Twitter, and Facebook, they might be encoded as 0, 1, and 2 respectively.

Output:
A new column Platform_encoded in the DataFrame.

Step 3:


Encode the 'Comment' Column using Count Vectorization.



Count Vectorization converts the text into a matrix of token counts. Each word in the text is represented as a feature.

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

# Count Vectorize the 'Comment' column
count_vectorizer = CountVectorizer(max_features=5000)
comments_count = count_vectorizer.fit_transform(data['Comment']).toarray()

# Convert to DataFrame
comments_count_df = pd.DataFrame(comments_count, columns=count_vectorizer.get_feature_names_out())

# Display the encoded comments data
print(comments_count_df.head())


   000  00am  02  10  100  1000001  100k  105  10days  10k  ...  zipped  \
0    0     0   0   0    0        0     0    0       0    0  ...       0   
1    0     0   0   0    0        0     0    0       0    0  ...       0   
2    0     0   0   0    0        0     0    0       0    0  ...       0   
3    0     0   0   0    0        0     0    0       0    0  ...       0   
4    0     0   0   0    0        0     0    0       0    0  ...       0   

   zipper  zoe  zombie  zone  zonked  zoology  zoomies  zuckerberg  äôt  
0       0    0       0     0       0        0        0           0    0  
1       0    0       0     0       0        0        0           0    0  
2       0    0       0     0       0        0        0           0    0  
3       0    0       0     0       0        0        0           0    0  
4       0    0       0     0       0        0        0           0    0  

[5 rows x 5000 columns]


Purpose:
To convert the text data into numerical features.

Count Vectorization:
Creates a matrix where each row represents a comment, and each column represents a word (feature) with the count of its occurrences.

max_features=5000:
 Limits the number of features to 5000 to manage computational complexity.

Output:
A DataFrame comments_count_df with word counts as features.

Step 4:


Combine Encoded Features.


We merge the encoded Platform column and the count vectorized Comment data into a single DataFrame.

In [4]:
# Combine all features into a single DataFrame
encoded_data = pd.concat([comments_count_df, data['Platform_encoded']], axis=1)
encoded_data['Hateful'] = data['Hateful']

# Display the combined encoded data
print(encoded_data.head())


   000  00am  02  10  100  1000001  100k  105  10days  10k  ...  zoe  zombie  \
0    0     0   0   0    0        0     0    0       0    0  ...    0       0   
1    0     0   0   0    0        0     0    0       0    0  ...    0       0   
2    0     0   0   0    0        0     0    0       0    0  ...    0       0   
3    0     0   0   0    0        0     0    0       0    0  ...    0       0   
4    0     0   0   0    0        0     0    0       0    0  ...    0       0   

   zone  zonked  zoology  zoomies  zuckerberg  äôt  Platform_encoded  Hateful  
0     0       0        0        0           0    0                 1        0  
1     0       0        0        0           0    0                 1        0  
2     0       0        0        0           0    0                 1        0  
3     0       0        0        0           0    0                 1        0  
4     0       0        0        0           0    0                 1        0  

[5 rows x 5002 columns]


Purpose:
To prepare the dataset for model training by combining all numerical features.

Concat:
Merges the DataFrames horizontally.

Output:
A DataFrame encoded_data with numerical features ready for machine learning.

Step 5:


Training and Evaluating a Machine Learning Model.


We use Logistic Regression, a widely used linear model for binary classification tasks. We split the data into training and testing sets, train the model, and evaluate its performance.

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Split the data into training and testing sets
X = encoded_data.drop('Hateful', axis=1)
y = encoded_data['Hateful']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Logistic Regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)


Accuracy: 0.9283333333333333
Classification Report:
              precision    recall  f1-score   support

           0       0.92      1.00      0.96       494
           1       0.97      0.61      0.75       106

    accuracy                           0.93       600
   macro avg       0.95      0.80      0.85       600
weighted avg       0.93      0.93      0.92       600



Purpose:
To build and evaluate a predictive model.

Logistic Regression:
A simple yet effective model for binary classification.

max_iter=1000:
Increases the maximum number of iterations for convergence.

Train-Test Split:
Divides the data into training (80%) and testing (20%) sets to evaluate model performance.

Metrics:

Accuracy:
Proportion of correctly predicted instances.

Classification Report:
Provides precision, recall, and F1-score for both classes (hateful and non-hateful).

why this method is effective:




Label Encoding:


Efficiently converts the categorical Platform column into numerical values without adding extra dimensions.


Count Vectorization:


Effectively captures the frequency of words in the Comment column, making it suitable for text classification tasks.


Logistic Regression:


A robust and simple linear model that works well with the encoded features.